# データセットクラスを書いてみよう

ここでは、Chainerにすでに用意されているCIFAR10のデータを取得する機能を使って、データセットクラスを自分で書いてみます。Chainerでは、データセットを表すクラスは以下の機能を持っていることが必要とされます。

- データセット内のデータ数を返す`__len__`メソッド
- 引数として渡される`i`に対応したデータもしくはデータとラベルの組を返す`get_example`メソッド

その他のデータセットに必要な機能は、`chainer.dataset.DatasetMixin`クラスを継承することで用意できます。ここでは、`DatasetMixin`クラスを継承した**Data augmentation**機能のついたデータセットクラスを作成してみましょう。

## 1. CIFAR10データセットクラスを書く

In [ ]:
import numpy as np
from chainer import dataset
from chainer.datasets import cifar

class CIFAR10(dataset.DatasetMixin):

    def __init__(self, train=True):
        train_data, test_data = cifar.get_cifar10()
        if train:
            self.data = train_data
        else:
            self.data = test_data
        self.train = train
        self.random_crop = 4

    def __len__(self):
        return len(self.data)

    def get_example(self, i):
        x, t = self.data[i]
        if self.train:
            x = x.transpose(1, 2, 0)
            h, w, _ = x.shape
            x_offset = np.random.randint(self.random_crop)
            y_offset = np.random.randint(self.random_crop)
            x = x[y_offset:y_offset + h - self.random_crop,
                  x_offset:x_offset + w - self.random_crop]
            if np.random.rand() > 0.5:
                x = np.fliplr(x)
            x = x.transpose(2, 0, 1)
        return x, t

このクラスは、CIFAR10のデータのそれぞれに対し、

- 32x32の大きさの中からランダムに28x28の領域をクロップ
- 1/2の確率で左右を反転させる

という加工を行っています。これによって、擬似的に学習データのバリエーションを増やすことができ、オーバーフィッティングを抑制することに役に立つことが知られています。

## 2. 作成したデータセットクラスを使って学習を行う

それではさっそくこの`CIFAR10`クラスを使って学習を行ってみましょう。以前使ったのと同じ畳み込み層のあるネットワークを使うことで、Data augmentationの効果がどの程度あるのかを調べてみましょう。

In [ ]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer.datasets import cifar
from chainer import iterators
from chainer import optimizers
from chainer import training
from chainer.training import extensions

# 前回と同じモデルを用意
class MyModel(chainer.Chain):
    
    def __init__(self, n_out):
        super(MyModel, self).__init__(
            conv1=L.Convolution2D(None, 32, 3, 3, 1),
            conv2=L.Convolution2D(32, 64, 3, 3, 1),
            conv3=L.Convolution2D(64, 128, 3, 3, 1),
            fc4=L.Linear(None, 1000),
            fc5=L.Linear(1000, n_out)
        )
        
    def __call__(self, x):
        h = F.relu(self.conv1(x))
        h = F.relu(self.conv2(h))
        h = F.relu(self.conv3(h))
        h = F.relu(self.fc4(h))
        h = self.fc5(h)
        return h

batchsize = 64
gpu_id = 0
max_epoch = 100

# 1. Dataset
train, test = CIFAR10(), CIFAR10(train=False)

# 2. Iterator
train_iter = iterators.SerialIterator(train, batchsize)
test_iter = iterators.SerialIterator(test, batchsize, False, False)

# 3. Model
model = MyModel(10)
model = L.Classifier(model)
model.to_gpu(gpu_id)

# 4. Optimizer
optimizer = optimizers.Adam()
optimizer.setup(model)

# 5. Updater
updater = training.StandardUpdater(train_iter, optimizer, device=gpu_id)

# 6. Trainer
trainer = training.Trainer(updater, (max_epoch, 'epoch'), out='cifar10_result')

trainer.extend(extensions.LogReport())
trainer.extend(extensions.Evaluator(test_iter, model, device=gpu_id))
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'main/accuracy', 'validation/main/loss', 'validation/main/accuracy', 'elapsed_time']))
trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], x_key='epoch', file_name='loss.png'))
trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'], x_key='epoch', file_name='accuracy.png'))
trainer.run()

epoch       main/loss   main/accuracy  validation/main/loss  validation/main/accuracy  elapsed_time
1           1.6131      0.408708       1.44266               0.476612                  5.72067       
2           1.34159     0.515965       1.41165               0.489749                  10.9183       
3           1.23198     0.559599       1.39705               0.505175                  16.1342       
4           1.1645      0.582366       1.32884               0.528264                  21.3852       
5           1.09811     0.607477       1.28315               0.544287                  27.0284       
6           1.05643     0.62452        1.28018               0.541401                  32.5634       
7           1.00531     0.641145       1.2171                0.568471                  37.8795       
8           0.968023    0.65649        1.21219               0.567874                  43.0586       
9           0.931772    0.667979       1.28827               0.548666               